In [ ]:
cfg_times = 3
cfg_lr = 0.02/4
cfg_classes = []
cfg_num_classes = 20
cfg_albu_p = 0.5
cfg_num_gpus = 2
cfg_mini_batch = 2
cfg_multi_scale = []
cfg_crop_size = 1280
cfg_load_from = None
cfg_experiment_path = './tmp/ipynbname'
cfg_train_data_root = '/workspace/notebooks/xxxx'
cfg_train_coco_file = 'keep_p_samples/01/train.json'
cfg_val_data_root = '/workspace/notebooks/xxxx'
cfg_val_coco_file = 'keep_p_samples/01/test.json'
cfg_test_data_root = '/workspace/notebooks/xxxx'
cfg_test_coco_file = 'keep_p_samples/01/test.json'
cfg_tmpl_path = '/usr/src/mmdetection/configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py'

In [ ]:
import os

#MMDET_PATH = '/usr/src/mmdetection'
#os.environ['MMDET_PATH'] = MMDET_PATH
#os.environ['MKL_THREADING_LAYER'] = 'GNU'

from cvtk.utils.notebook import clean_models

In [ ]:
%%time
albu_train_transforms = [
    dict(type='RandomRotate90', p=cfg_albu_p),
]
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='Albu',
        transforms=albu_train_transforms,
        bbox_params=dict(
            type='BboxParams',
            format='pascal_voc',
            label_fields=['gt_labels'],
            min_visibility=0.0,
            filter_lost_elements=True),
        keymap={
            'img': 'image',
            'gt_masks': 'masks',
            'gt_bboxes': 'bboxes'
        },
        update_pad_shape=False,
        skip_img_without_anno=True),
    dict(type='Resize', test_mode=(len(cfg_multi_scale) == 0), multi_scale=cfg_multi_scale),
    dict(type='RandomCrop', height=cfg_crop_size, width=cfg_crop_size),
    dict(type='RandomFlip', flip_ratio=0.5, direction=['horizontal', 'vertical', 'diagonal']),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels']),
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        scale_factor=1.0,
        transforms=[
            dict(type='Resize', test_mode=True, multi_scale=cfg_multi_scale),
            dict(type='Normalize', **img_norm_cfg),
            dict(type='Pad', size_divisor=32),
            dict(type='DefaultFormatBundle'),
            dict(type='Collect', keys=['img']),
        ]),
]

cfg_data = dict(
    samples_per_gpu=cfg_mini_batch,
    workers_per_gpu=cfg_mini_batch,
    train=dict(
        type='CocoDataset',
        ann_file=os.path.join(cfg_train_data_root, cfg_train_coco_file),
        classes=cfg_classes,
        img_prefix=cfg_train_data_root,
        pipeline=train_pipeline),
    val=dict(
        type='CocoDataset',
        ann_file=os.path.join(cfg_val_data_root, cfg_val_coco_file),
        classes=cfg_classes,
        img_prefix=cfg_val_data_root,
        pipeline=test_pipeline),
    test=dict(
        type='CocoDataset',
        ann_file=os.path.join(cfg_test_data_root, cfg_test_coco_file),
        classes=cfg_classes,
        img_prefix=cfg_test_data_root,
        pipeline=test_pipeline))

cfg_model = dict(
    type='FasterRCNN',
    pretrained='torchvision://resnet50',
    backbone=dict(
        type='ResNet',
        depth=50,
    ),
    neck=dict(
        num_outs=5,
        start_level=1,
        add_extra_convs='on_input',
        relu_before_extra_convs=False,
    ),
    rpn_head=dict(
        anchor_generator=dict(
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[8, 16, 32, 64, 128],
        ),
    ),
    roi_head=dict(
        bbox_roi_extractor=dict(
            featmap_strides=[8, 16, 32, 64, 128],
            finest_scale=56,
        ),
        bbox_head=dict(
            num_classes=cfg_num_classes,
        ),
    ),
)

cfg_lr_config = dict(
    _delete_=True,
    policy='Step',
    step=[8 * cfg_times, 11 * cfg_times],
    gamma=0.1,
    warmup='linear',
    warmup_iters=1500,
    warmup_ratio=0.001,
)

cfg_log_config = dict(
    interval=50,
    hooks=[
        dict(type='TextLoggerHook'),
    ],
)

cfg_options = dict(
    optimizer=dict(type='SGD', lr=cfg_lr, momentum=0.9, weight_decay=0.0001),
    runner=dict(type='EpochBasedRunner', max_epochs=12 * cfg_times),
    evaluation=dict(interval=2, metric='bbox'),
    checkpoint_config=dict(interval=1),
    log_config=cfg_log_config,
    lr_config=cfg_lr_config,
    load_from=cfg_load_from,
    model=cfg_model,
    data=cfg_data)
os.environ['CFG_OPTIONS'] = str(cfg_options)

ARG_TRAIN = '{} --work-dir {} --launcher pytorch'.format(cfg_tmpl_path, cfg_experiment_path)
!python -m torch.distributed.launch --nproc_per_node={cfg_num_gpus} dev/py_train.py {ARG_TRAIN}
DEL_FILES = ' '.join(clean_models(cfg_experiment_path, 3))
logs = !rm -rfv {DEL_FILES}
cfg_experiment_path

In [ ]:
%%time
import os

work_dir = cfg_experiment_path
config = os.path.basename(cfg_tmpl_path)

data_root = cfg_test_data_root#'/workspace/notebooks/xxxx'
coco_file = cfg_test_coco_file#'keep_p_samples/01/train.json'
gpus = cfg_num_gpus
config_file = os.path.join(work_dir, config)
checkpoint_file = os.path.join(work_dir, 'latest.pth')
batch_size = 1
workers_per_gpu = 2

ARG_TEST = f'{data_root} {coco_file} {gpus} {config_file} {checkpoint_file} {batch_size} {workers_per_gpu}'
logs = !python dev/py_test.py {ARG_TEST}
print('\n'.join(logs))

from cvtk.utils.abc.discover import hardmini_test
[hardmini_test(logs, level='image', score=s, nok=True) for s in (0.3, 0.5, 0.85, 1.01)]